# Setup AWS Credentials in EKS cluster

AWS credentials are required to save a model in S3. These credentials are stored in the EKS cluster as a Kubernetes secret.

Create an IAM user `s3user`, attach the S3 access policy, and retrieve temporary credentials

In [1]:
%%bash

source ~/.bash_profile

aws iam create-user --user-name s3user

{
    "User": {
        "Path": "/",
        "UserName": "s3user",
        "UserId": "AIDAVFCCPI2LHQZJXZMVZ",
        "Arn": "arn:aws:iam::354474215062:user/s3user",
        "CreateDate": "2020-09-26T21:13:13Z"
    }
}


In [2]:
%%bash

source ~/.bash_profile

aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
aws iam attach-user-policy --user-name s3user --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

In [3]:
%%bash

source ~/.bash_profile

aws iam create-access-key --user-name s3user > /tmp/create_output.json

# Create `aws-secret` to Allow Kubernetes/Kubeflow to Access Amazon SageMaker

In [4]:
%%bash

source ~/.bash_profile

export AWS_ACCESS_KEY_ID_VALUE=$(jq -j .AccessKey.AccessKeyId /tmp/create_output.json | base64)
echo "export AWS_ACCESS_KEY_ID_VALUE=${AWS_ACCESS_KEY_ID_VALUE}" | tee -a ~/.bash_profile

export AWS_SECRET_ACCESS_KEY_VALUE=$(jq -j .AccessKey.SecretAccessKey /tmp/create_output.json | base64)
echo "export AWS_SECRET_ACCESS_KEY_VALUE=${AWS_SECRET_ACCESS_KEY_VALUE}" | tee -a ~/.bash_profile

cat <<EOF | kubectl apply --namespace kubeflow -f -
apiVersion: v1
kind: Secret
metadata:
  name: aws-secret
type: Opaque
data:
  AWS_ACCESS_KEY_ID: $AWS_ACCESS_KEY_ID_VALUE
  AWS_SECRET_ACCESS_KEY: $AWS_SECRET_ACCESS_KEY_VALUE
EOF

export AWS_ACCESS_KEY_ID_VALUE=QUtJQVZGQ0NQSTJMTERHR1VPTkg=
export AWS_SECRET_ACCESS_KEY_VALUE=UVdWd3I3aGVCWmxUdHNmVndaVWd0WmRzQ0phM1RrNzhsN2dTNm1mTQ==
secret/aws-secret created


# Add More Roles/Policies for Kubernetes/Kubeflow to Access SageMaker

In [6]:
%%bash

source ~/.bash_profile

TRUST="{ \"Version\": \"2012-10-17\", \"Statement\": [ { \"Effect\": \"Allow\", \"Principal\": { \"Service\": \"sagemaker.amazonaws.com\" }, \"Action\": \"sts:AssumeRole\" } ] }"
aws iam create-role --role-name workshop-sagemaker-kfp-role --assume-role-policy-document "$TRUST"
aws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess

export SAGEMAKER_ROLE_ARN=$(aws iam get-role --role-name workshop-sagemaker-kfp-role --output text --query 'Role.Arn')
echo "export SAGEMAKER_ROLE_ARN=${SAGEMAKER_ROLE_ARN}" | tee -a ~/.bash_profile

cat <<EoF > sagemaker-invoke.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}
EoF

aws iam put-role-policy --role-name workshop-sagemaker-kfp-role --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-invoke.json
aws iam put-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-invoke.json

export SAGEMAKER_ROLE_ARN=arn:aws:iam::354474215062:role/workshop-sagemaker-kfp-role



An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name workshop-sagemaker-kfp-role already exists.
Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


CalledProcessError: Command 'b'\nsource ~/.bash_profile\n\nTRUST="{ \\"Version\\": \\"2012-10-17\\", \\"Statement\\": [ { \\"Effect\\": \\"Allow\\", \\"Principal\\": { \\"Service\\": \\"sagemaker.amazonaws.com\\" }, \\"Action\\": \\"sts:AssumeRole\\" } ] }"\naws iam create-role --role-name workshop-sagemaker-kfp-role --assume-role-policy-document "$TRUST"\naws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess\naws iam attach-role-policy --role-name workshop-sagemaker-kfp-role --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess\n\nexport SAGEMAKER_ROLE_ARN=$(aws iam get-role --role-name workshop-sagemaker-kfp-role --output text --query \'Role.Arn\')\necho "export SAGEMAKER_ROLE_ARN=${SAGEMAKER_ROLE_ARN}" | tee -a ~/.bash_profile\n\ncat <<EoF > sagemaker-invoke.json\n{\n    "Version": "2012-10-17",\n    "Statement": [\n        {\n            "Effect": "Allow",\n            "Action": [\n                "sagema

In [7]:
!cat sagemaker-invoke.json

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}


In [1]:
%%bash

source ~/.bash_profile

cat <<EoF > sagemaker-full.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:*"
            ],
            "Resource": "*"
        }
    ]
}
EoF

aws iam put-role-policy --role-name workshop-sagemaker-kfp-role --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-full.json
aws iam put-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-full.json

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


CalledProcessError: Command 'b'\nsource ~/.bash_profile\n\ncat <<EoF > sagemaker-full.json\n{\n    "Version": "2012-10-17",\n    "Statement": [\n        {\n            "Effect": "Allow",\n            "Action": [\n                "sagemaker:*"\n            ],\n            "Resource": "*"\n        }\n    ]\n}\nEoF\n\naws iam put-role-policy --role-name workshop-sagemaker-kfp-role --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-full.json\naws iam put-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-name sagemaker-invoke-for-worker --policy-document file://sagemaker-full.json\n'' returned non-zero exit status 2.

In [9]:
!cat sagemaker-full.json

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:*"
            ],
            "Resource": "*"
        }
    ]
}


# Add More Roles/Policies for Kubernetes/Kubeflow to Access S3

In [10]:
%%bash

source ~/.bash_profile

aws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess
aws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument
Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


CalledProcessError: Command 'b'\nsource ~/.bash_profile\n\naws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess\naws iam attach-role-policy --role-name ${INSTANCE_ROLE_NAME} --policy-arn arn:aws:iam::aws:policy/AmazonSageMakerFullAccess\n'' returned non-zero exit status 2.

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();